In [1]:
import optuna
import os
from pathlib import Path
import kaggle
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 5
OUTPUT = f'cat_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(ColumnTransformer([('drop', 'drop',
                                  ['iv(g)', 't', 'b', 'n',
                                   'lOCode', 'v', 'branchCount',
                                   'e', 'i', 'lOComment'])],
                                remainder='passthrough'), SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators',50,1000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.7,log=True),
        'depth': trial.suggest_int('depth',1, 12, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8, 100,log=True),
        'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
        'random_strength': trial.suggest_float('random_strength',1e-8,10,log=True),
        'subsample': trial.suggest_float("subsample", 0.1, 1),
        'verbose': trial.suggest_categorical('verbose', [False])
    }
    pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-15 23:48:43,218] A new study created in memory with name: no-name-38360b1d-8643-4f5c-a844-bcedf12d92f3
Best trial: 0. Best value: 0.785131:   3%|█████▍                                                                                                                                                              | 1/30 [00:09<04:49,  9.98s/it]

[I 2023-10-15 23:48:53,198] Trial 0 finished with value: 0.7851305010976195 and parameters: {'n_estimators': 84, 'learning_rate': 0.036288549427801804, 'depth': 9, 'l2_leaf_reg': 0.0006792275201233925, 'model_size_reg': 0.00038303754039768024, 'random_strength': 2.304579022122091e-08, 'subsample': 0.49865001146905263, 'verbose': False}. Best is trial 0 with value: 0.7851305010976195.


Best trial: 0. Best value: 0.785131:   7%|██████████▉                                                                                                                                                         | 2/30 [00:41<10:36, 22.72s/it]

[I 2023-10-15 23:49:24,830] Trial 1 finished with value: 0.7839258705247862 and parameters: {'n_estimators': 477, 'learning_rate': 0.16390018990311508, 'depth': 4, 'l2_leaf_reg': 2.4050997923277563e-08, 'model_size_reg': 3.507379710004656, 'random_strength': 8.196811352009398e-06, 'subsample': 0.1354472230353554, 'verbose': False}. Best is trial 0 with value: 0.7851305010976195.


Best trial: 0. Best value: 0.785131:  10%|████████████████▍                                                                                                                                                   | 3/30 [00:53<08:02, 17.86s/it]

...

[I 2023-10-15 23:56:05,431] Trial 27 finished with value: 0.7837514698844639 and parameters: {'n_estimators': 109, 'learning_rate': 0.11670413273149334, 'depth': 10, 'l2_leaf_reg': 0.005763545630530416, 'model_size_reg': 7.503852286957007, 'random_strength': 6.6178244666951625e-06, 'subsample': 0.5001954604852553, 'verbose': False}. Best is trial 24 with value: 0.7856567284821686.


Best trial: 24. Best value: 0.785657:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [07:30<00:08,  8.87s/it]

[I 2023-10-15 23:56:13,687] Trial 28 finished with value: 0.7841105693542609 and parameters: {'n_estimators': 75, 'learning_rate': 0.2764030876496122, 'depth': 7, 'l2_leaf_reg': 0.038416031515949446, 'model_size_reg': 0.2663932413213687, 'random_strength': 4.361494541266555e-05, 'subsample': 0.36979582940955247, 'verbose': False}. Best is trial 24 with value: 0.7856567284821686.


Best trial: 24. Best value: 0.785657: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [07:41<00:00, 15.38s/it]

[I 2023-10-15 23:56:24,562] Trial 29 finished with value: 0.7852399194673245 and parameters: {'n_estimators': 84, 'learning_rate': 0.044896498538627594, 'depth': 9, 'l2_leaf_reg': 0.008603379022941794, 'model_size_reg': 93.79741835622181, 'random_strength': 6.997844407359394e-08, 'subsample': 0.48105081919088627, 'verbose': False}. Best is trial 24 with value: 0.7856567284821686.


In [7]:
study.best_params, study.best_value

({'n_estimators': 68,
  'learning_rate': 0.12029161791941848,
  'depth': 6,
  'l2_leaf_reg': 0.009418069893643286,
  'model_size_reg': 7.007768945351371,
  'random_strength': 1.8341119786271264e-05,
  'subsample': 0.6007074418762239,
  'verbose': False},
 0.7856567284821686)

In [8]:
pipeline = make_pipeline(partial_pipeline, CatBoostClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['iv(g)',
                                                                    't', 'b',
                                                                    'n',
                                                                    'lOCode',
                                                                    'v',
                                                                    'branchCount',
                                                                    'e', 'i',
                                                                    'lOComment'])])),
                                 ('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x7f2301564f50>)])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)